In [1]:
%%loadFromPOM
<dependency>
    <groupId>org.apache.poi</groupId>
    <artifactId>poi-ooxml</artifactId>
    <version>5.1.0</version>
</dependency>

In [2]:
import java.io.File;
import java.io.IOException;
import java.time.LocalDateTime;
import java.time.LocalDate;
import java.time.Duration;
import java.time.format.DateTimeFormatter;
import java.time.ZoneId;

import org.apache.poi.ss.usermodel.*;
import org.apache.poi.xssf.usermodel.*;


In [3]:
LocalDateTime tiempoInicio = LocalDateTime.now();
DateTimeFormatter formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd HH:mm:ss");
System.out.println("[" + tiempoInicio.format(formatter) + "] Comienzo de la validación de datos");

// Variables globales
String PATH = System.getProperty("user.dir");
String INPUT_FOLDER = "input";
String AUX_FOLDER = "aux_file";
String INPUT_FILENAME = "dataset.xlsx";
String OUTPUT_FILENAME = "dataset_validado.xlsx";
boolean ERROR = false;

// Si no está la carpeta aux_file, la creo
File auxFolder = new File(PATH + "/" + AUX_FOLDER);
if (!auxFolder.exists()) {
    auxFolder.mkdir();
}

// Cargo los datos
try {
    FileInputStream file = new FileInputStream(new File(PATH + "/" + INPUT_FOLDER + "/" + INPUT_FILENAME));
    XSSFWorkbook workbook = new XSSFWorkbook(file);
    XSSFSheet sheet = workbook.getSheetAt(0);

    System.out.println("[" + LocalDateTime.now().format(formatter) + "] El archivo se cargó exitosamente");
    

    // Validación del campo Fecha
    DateTimeFormatter dateFormatterInput = DateTimeFormatter.ofPattern("dd-MMM-yyyy", Locale.ENGLISH);
    DateTimeFormatter dateFormatterOutput = DateTimeFormatter.ofPattern("yyyy-MM-dd");
    for (Row row : sheet) {
        if (row.getRowNum() == 0) {
            continue; // Saltear la primera fila de encabezado
        }
        Cell fechaCell = row.getCell(0);
        if (fechaCell.getCellType() == CellType.NUMERIC) {
            LocalDate fecha = DateUtil.getJavaDate(fechaCell.getNumericCellValue()).toInstant().atZone(ZoneId.systemDefault()).toLocalDate();
            fechaCell.setCellValue(fecha.format(dateFormatterInput)); // Formateo la fecha para asegurarme de que tenga el formato correcto
        } else if (fechaCell.getCellType() == CellType.STRING) {
            try {
                LocalDate fecha = LocalDate.parse(fechaCell.getStringCellValue(), dateFormatterInput);
                fechaCell.setCellValue(fecha.format(dateFormatterInput)); // Formateo la fecha para asegurarme de que tenga el formato correcto
            } catch (Exception e) {
                System.out.println("[" + LocalDateTime.now().format(formatter) + "] El campo Fecha no se encuentra cargado correctamente");
                ERROR = true;
                break;
            }
        }
    }
    if (!ERROR) {
        FileOutputStream outputStream = new FileOutputStream(PATH + "/" + AUX_FOLDER + "/" + OUTPUT_FILENAME);
        workbook.write(outputStream);
        outputStream.close();
        System.out.println("[" + LocalDateTime.now().format(formatter) + "] El archivo fue validado existosamente");

        LocalDateTime tiempoFin = LocalDateTime.now();
        Duration duracion = Duration.between(tiempoInicio, tiempoFin);
        long horas = duracion.toHours();
        duracion = duracion.minusHours(horas);
        long minutos = duracion.toMinutes();
        duracion = duracion.minusMinutes(minutos);
        long segundos = duracion.getSeconds();
        System.out.printf("[%s] El tiempo transcurrido de ejecución fue: %02dh %02dm %02ds\n", LocalDateTime.now().format(formatter), horas, minutos, segundos);
    }
    workbook.close();
} catch (IOException e) {
    System.out.println("[" + LocalDateTime.now().format(formatter) + "] Se produjo un error en la carga de archivo: " + e.getMessage());
    ERROR = true;
}

if (ERROR) {
    System.exit(0);
}

[2023-04-02 23:10:38] Comienzo de la validación de datos


ERROR StatusLogger Log4j2 could not find a logging implementation. Please add log4j-core to the classpath. Using SimpleLogger to log to the console...


[2023-04-02 23:10:39] El archivo se cargó exitosamente
[2023-04-02 23:10:39] El archivo fue validado existosamente
[2023-04-02 23:10:39] El tiempo transcurrido de ejecución fue: 00h 00m 01s
